In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from scipy import optimize, linalg
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

**Systems of Linear Equations**

$n$개의 선형 연립방정식에 대해서,

$$\begin{cases}
A_{11}x_1 + A_{12}x_2 + \cdots + A_{1n}x_n &= b_1 \\
A_{21}x_1 + A_{22}x_2 + \cdots + A_{2n}x_n &= b_2 \\
&\vdots \\
A_{n1}x_1 + A_{n2}x_2 + \cdots + A_{nn}x_n &= b_n \\
\end{cases}$$

행렬을 이용해 나타낼 수 있다.
$$\begin{pmatrix}
A_{11} & A_{12} & \cdots & A_{1n} \\
A_{21} & A_{22} & \cdots & A_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
A_{n1} & A_{n2} & \cdots & A_{nn} \\
\end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{pmatrix}
= \begin{pmatrix} b_1 \\ b_2 \\ \vdots \\ b_n \end{pmatrix}
\quad\Leftrightarrow\quad \boxed{\mathbf{A\,x} = \mathbf{b}}$$

**[Example]** 4개의 식으로 이루어진 선형 연립방정식의 해을 구하시오.
$$\begin{cases}
10x_1 + 2x_2 &= 3 \\
3x_1 + 10x_2 + 4 x_3 &=4 \\
x_2 + 7x_3 + 5x_4 &=5 \\
3x_3 + 4x_4 &= 6
\end{cases}
\quad\Leftrightarrow\quad
\begin{pmatrix}
10 & 2 & 0 & 0 \\
3 & 10 & 4 & 0 \\
0 & 1 & 7 & 5 \\
0 & 0 & 3 & 4 
\end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{pmatrix}
=
\begin{pmatrix} 3 \\ 4 \\ 5 \\ 6 \end{pmatrix}$$

In [0]:
# 4x4 Matrix
mat = np.array([[10,2,0,0], [3,10,4,0], [0,1,7,5], [0,0,3,4]])
rhs = np.array([3,4,5,6])
print(mat, rhs)

[[10  2  0  0]
 [ 3 10  4  0]
 [ 0  1  7  5]
 [ 0  0  3  4]] [3 4 5 6]


In [0]:
# Inverse matrix
linalg.inv(mat).dot(rhs)

array([ 0.14877589,  0.75612053, -1.00188324,  2.25141243])

# Direct Matrix Solvers

## `scipy.linalg.solve`

```python
linalg.solve(a, b, sym_pos=False, lower=False, overwrite_a=False, 
        overwrite_b=False, debug=None, check_finite=True, assume_a='gen', 
        transposed=False)
```

In [0]:
# General matrix solver
linalg.solve(mat, rhs)

array([ 0.14877589,  0.75612053, -1.00188324,  2.25141243])

## `scipy.linalg.lu_solve`

```python
linalg.lu_solve(lu_and_piv, b, trans=0, overwrite_b=False, check_finite=True)
```

In [0]:
# LU matrix solver
lu = linalg.lu_factor(mat)
linalg.lu_solve(lu, rhs)

array([ 0.14877589,  0.75612053, -1.00188324,  2.25141243])

## `scipy.linalg.solve_banded`

```python
linalg.solve_banded((l, u), ab, b, overwrite_ab=False, overwrite_b=False, 
            debug=None, check_finite=True)
```

In [0]:
# Banded matrix solver
mat = np.empty((3, 4))
mat[0] = np.r_[0, 2, 4, 5]
mat[1] = np.r_[10, 10, 7, 4]
mat[2] = np.r_[3, 1, 3, 0]

linalg.solve_banded((1,1), mat, rhs)

array([ 0.14877589,  0.75612053, -1.00188324,  2.25141243])

## `scipy.sparse.linalg.spsolve`#

```python
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

spdiags(data, diags, m, n, format=None)
spsolve(A, b, permc_spec=None, use_umfpack=True)
```

In [0]:
# Sparse matrix solver: spsolve
udiag = np.r_[0, 2, 4, 5]
cdiag = np.r_[10, 10, 7, 4]
ldiag = np.r_[3, 1, 3, 0]
mat = spdiags((udiag, cdiag, ldiag), [1, 0, -1], 4, 4, format="csr")
spsolve(mat, rhs)

array([ 0.14877589,  0.75612053, -1.00188324,  2.25141243])

# Iterative Matrix Solvers

3x3 행렬에 대해, 행렬식 $Ax = b$로 나타낼 수 있다.
$$\begin{cases}
a_{11}x_1 + a_{12}x_2 + a_{13}x_3 = b_1 \\
a_{21}x_1 + a_{22}x_2 + a_{23}x_3 = b_2 \\
a_{31}x_1 + a_{32}x_2 + a_{33}x_3 = b_3
\end{cases}\quad\Leftrightarrow\quad
\begin{pmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} 
\end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix}=
\begin{pmatrix} b_1 \\ b_2 \\ b_3 \end{pmatrix}$$

행렬 $A$를 대각행렬 $D$, 하단 삼각행렬 $L$, and 상단 삼각행렬 $U$의 합으로 나타내면, $A = U + D + L$ 이고,
$$Ax = b \quad\Leftrightarrow\quad (L+D+U)x = b$$

개별 변수값을 순차적으로 구하기 위해 다음과 같이 변형할 수 있다.
$$\begin{cases}
a_{11}x_1 = b_1 \phantom{- a_{11}x_1} - a_{12}x_2 - a_{13}x_3 \\
a_{22}x_2 = b_2 - a_{21}x_1 \phantom{- a_{22}x_2} - a_{23}x_3 \\
a_{33}x_3 = b_3 - a_{31}x_1 - a_{32}x_2 \phantom{- a_{33}x_3}
\end{cases}\quad\Leftrightarrow\quad Dx = b - Lx - Ux$$

행렬로 나타내면,
$$\begin{bmatrix} a_{11} \\ & a_{22} \\ & & a_{33}
\end{bmatrix}\begin{bmatrix}x_1 \\ x_2 \\ x_3 \end{bmatrix}
= \begin{bmatrix}b_1 \\ b_2 \\ b_3 \end{bmatrix} 
- \begin{bmatrix} 0 & 0 & 0 \\ a_{21} & 0 & 0 \\ a_{31} & a_{32} & 0
\end{bmatrix}\begin{bmatrix}x_1 \\ x_2 \\ x_3 \end{bmatrix}
- \begin{bmatrix} 0 & a_{12} & a_{13} \\ 0 & 0 & a_{23} \\ 0 & 0 & 0
\end{bmatrix}\begin{bmatrix}x_1 \\ x_2 \\ x_3 \end{bmatrix}
$$

**Convergence of Iterative Methods:**

행렬 $T$에 대해 $x^{k+1} = T x^{k} + c, (k\ge1)$ 일 때,
$$x^{k+1} = T x^{k} + c = T(T x^{k-1} + c) + c = T^k x^0 + (T^{k-1} + \cdots + T + I)c$$

$||T||<1$이면, $\lim_{k\to\infty}T^k x^0 = 0$ 이므로,
$$\lim_{k\to\infty} x^k = \lim_{k\to\infty}\left[\sum_{j=0}^{k-1}T^j\right]c =(1-T)^{-1}c$$

따라서 $x$는 $(1-T)^{-1}$에 수렴한다.

- Jacobi's method: $x^{k+1} = T_Jx^{k} + c_J$, $T_J = D^{-1}(L+U)$, $c_J = D^{-1}b$

- Gauss-Seidel method: $x^{k+1} = T_{GS}x^{k} + c_{GS}$, $T_{GS} = (D+L)^{-1}U$, $c_{GS} = (D+L)^{-1}b$

In [0]:
# 4x4 Matrix
mat = np.array([[10,2,0,0], [3,10,4,0], [0,1,7,5], [0,0,3,4]])
rhs = np.array([3,4,5,6])

print(mat, rhs)

[[10  2  0  0]
 [ 3 10  4  0]
 [ 0  1  7  5]
 [ 0  0  3  4]] [3 4 5 6]


## Jacobi's Method

$\mathbf{x}^{k+1}=[x_1^{k+1}, x_2^{k+1}, x_3^{k+1}]$을 이전에 계산된 $\mathbf{x}^k = [x_1^{k}, x_2^{k}, x_3^{k}]$를 이용하여 업데이트 하면,
$$\begin{cases}
a_{11}x_1^{k+1} = b_1 \phantom{- a_{11}x_1^{k}} - a_{12}x_2^{k} - a_{13}x_3^{k} \\
a_{22}x_2^{k+1} = b_2 - a_{21}x_1^{k} \phantom{- a_{22}x_2^{k}} - a_{23}x_3^{k} \\
a_{33}x_3^{k+1} = b_3 - a_{31}x_1^{k} - a_{32}x_2^{k} \phantom{- a_{33}x_3^{k}}
\end{cases}\quad\Leftrightarrow\quad Dx^{k+1} = b - Lx^k - Ux^k$$

대각행렬 $D$의 역행렬을 양변에 곱하면,
\begin{align*}
x^{k+1} &= D^{-1}\left(b - Lx^k - Ux^k\right) \\
x_i^{k+1} &= \frac{1}{a_{ii}}\left(b_i - \sum_{j=1}^{i-1} a_{ij}x^k - \sum_{j=i+1}^n a_{ij}x_j^k\right) 
\end{align*}

**Code-1**: `for`문을 이용한 행렬식 계산

```python
x2 = np.zeros_like(x1)
for i in range(x1.size):
    lower = upper = 0
    for j in range(i):
        lower += A[i,j]*x1[j]
    for j in range(i+1, x1.size):
        upper += A[i,j]*x1[j]
    x2[i] = (b[i] - lower - upper)/A[i,i]
```

**Code-2**: 벡터연산을 이용한 행렬식 계산

```python
x2 = np.zeros_like(x1)
for i in range(x1.size):
    lower = A[i,:i].dot(x1[:i])
    upper = A[i,i+1:].dot(x1[i+1:])
    x2[i] = (b[i] - lower - upper)/A[i,i]
```

**Code-3**: 행렬을 이용한 계산
```python
diag = np.diag(A)
L = np.tril(A) - np.diag(diag)
U = np.triu(A) - np.diag(diag)
x2 = (b - L.dot(x1) - U.dot(x1))/diag
```

In [0]:
def jacobi(A, b, x0, maxiter=1000):
    def update(x1):
        x2 = np.zeros_like(x1)
        for i in range(x1.size):
            lower = A[i,:i].dot(x1[:i]) # Update x2 using x1
            upper = A[i,i+1:].dot(x1[i+1:])
            x2[i] = (b[i] - lower - upper)/A[i,i]
        return x2

    x1 = np.asfarray(x0)
    for k in range(maxiter):
        x2 = update(x1)
        if np.allclose(x2, x1): break
        else: x1 = x2
    return x2, k+1

jacobi(mat, rhs, [0, 0, 0, 0])

(array([ 0.14877784,  0.75611719, -1.00187015,  2.25140681]), 47)

## Gauss-Seidel Method

$\mathbf{x}^{k+1}=[x_1^{k+1}, x_2^{k+1}, x_3^{k+1}]$을 이전에 계산된 $\mathbf{x}^k = [x_1^{k}, x_2^{k}, x_3^{k}]$를 이용하여 업데이트 하면서, 업데이드된 일부 $\mathbf{x}^{k+1}$ 값들도 동시에 이용하면, 
$$\begin{cases}
a_{11}x_1^{k+1} = b_1 - \phantom{a_{11}x_1^{k+1}} - a_{12}x_2^{k}\phantom{1} - a_{13}x_3^{k} \\
a_{22}x_2^{k+1} = b_2 - a_{21}x_1^{k+1} \phantom{- a_{22}x_2^{k+1}} - a_{23}x_3^{k} \\
a_{33}x_3^{k+1} = b_3 - a_{31}x_1^{k+1} - a_{32}x_2^{k+1} \phantom{- a_{33}x_3^{k}}
\end{cases}\quad\Leftrightarrow\quad Dx^{k+1} = b - Lx^{k+1} - Ux^k$$

대각행렬 $D$의 역행렬을 양변에 곱하면,
\begin{align*}
x^{k+1} &= D^{-1}\left(b - Lx^{k+1} - Ux^k\right) \\
x_i^{k+1} &= \frac{1}{a_{ii}}\left(b_i - \sum_{j=1}^{i-1} a_{ij}x^{k+1} - \sum_{j=i+1}^n a_{ij}x_j^k\right)
\end{align*}

In [0]:
def gauss_seidel(mat, rhs, x0, maxiter=1000):
    def update(x1):
        x2 = np.zeros_like(x1)
        for i in range(x1.size):
            lower = mat[i,:i].dot(x2[:i]) # Update x2 using updated x2
            upper = mat[i,i+1:].dot(x1[i+1:])
            x2[i] = (rhs[i] - lower - upper)/mat[i,i]
        return x2

    x1 = np.asfarray(x0)
    for k in range(maxiter):
        x2 = update(x1)
        if np.allclose(x2, x1): break
        else: x1 = x2
    return x2, k+1

gauss_seidel(mat, rhs, [0, 0, 0, 0])

(array([ 0.14877778,  0.75611487, -1.00187562,  2.25140671]), 25)

## Successive Over-relaxation (SOR) Method

Gauss-Seidel Method에서
$$x^{k+1} = D^{-1}\left(b - Lx^{k+1} - Ux^k\right)$$

Relaxation factor $\omega>1$에 대해,
\begin{align*}
x^{k+1} &= (1-\omega)x^k + \omega D^{-1}\left(b - Lx^{k+1} - Ux^k\right)\\
x_i^{k+1} &= (1-\omega)x_i^k + \frac{\omega}{a_{ii}}\left(b_i - \sum_{j=1}^{i-1} a_{ij}x^{k+1} - \sum_{j=i+1}^n a_{ij}x_j^k\right)
\end{align*}

In [0]:
def sor(A, b, x0, omega=1.0, maxiter=1000):
    def update(x1):
        x2 = np.zeros_like(x1)
        for i in range(x1.size):
            lower = A[i,:i].dot(x2[:i]) # Update x2 using updated x2
            upper = A[i,i+1:].dot(x1[i+1:])
            x2[i] = (b[i] - lower - upper)/A[i,i]
        return (1-omega)*x1 + omega*x2

    x1 = np.asfarray(x0)
    for k in range(maxiter):
        x2 = update(x1)
        if np.allclose(x2, x1): break
        else: x1 = x2
    return x2, k+1

sor(mat, rhs, [0, 0, 0, 0], omega=1.3)

(array([ 0.14877649,  0.7561188 , -1.0018809 ,  2.25141068]), 19)